# Hourly ridership prediction

In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/thehari08/Work/surya/surya_previous/lively-encoder-448916-d5-5e9819f4df4e.json"

from datetime import datetime

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql import Window
from pyspark.sql.functions import col, count, countDistinct, sum, min, max, avg, stddev, hour, dayofweek, dayofmonth, ceil, sin, cos, lit, pi, lag, unix_timestamp, create_map
from pyspark.sql.types import (
    StructType, StructField, StringType, LongType, DoubleType, ArrayType, TimestampType, FloatType
)

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import sklearn
import xgboost as xgb
from xgboost.spark import SparkXGBRegressor
from pyspark.ml.regression import RandomForestRegressor

### Initialize Spark session and import data from BigQuery

In [2]:
# Stop if existing Spark session is running
if "spark" in locals():
    spark.stop()

In [3]:
# Initialize Spark session with GCS configuration
spark = SparkSession.builder \
    .appName("NYC Subway Data Processing") \
    .config("spark.jars", 
            "/home/thehari08/Work/surya/surya_previous/gcs-connector-hadoop3-latest.jar, /home/thehari08/Work/surya/surya_previous/spark-bigquery-with-dependencies_2.12-0.28.0.jar, https://repo1.maven.org/maven2/ml/dmlc/xgboost4j-spark_2.13/2.1.4/xgboost4j-spark_2.13-2.1.4.jar") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", 
            "/home/thehari08/Work/surya/surya_previous/lively-encoder-448916-d5-5e9819f4df4e.json") \
    .master("local[24]") \
    .config("spark.driver.memory", "48g") \
    .config("spark.executor.memory", "48g") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.executor.cores", "6") \
    .config("spark.task.cpus", "1") \
    .config("spark.default.parallelism", "56") \
    .config("spark.sql.shuffle.partitions", "56") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.rdd.compress", "true") \
    .config("spark.locality.wait", "0") \
    .config("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED") \
    .config("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED") \
    .config("spark.sql.parquet.enableVectorizedReader", "false") \
    .config("spark.sql.execution.arrow.enabled", "false") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
    .config("spark.sql.execution.arrow.fallback.enabled", "false") \
    .config("spark.local.dir", "/home/thehari08/Work/spark") \
    .getOrCreate()

# Suppress warnings, show only errors
spark.sparkContext.setLogLevel("ERROR")

25/04/18 09:37:48 WARN Utils: Your hostname, theBEAST resolves to a loopback address: 127.0.1.1; using 10.20.30.10 instead (on interface enp4s0)
25/04/18 09:37:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/04/18 09:37:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/18 09:37:51 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [4]:
# Import data from BigQuery
gcs_bucket_path = "gs://hour_pred_data/*.parquet"
df = spark.read.parquet(gcs_bucket_path)

df.show(1, truncate=False)
df.count()

+-------------------+---------+--------+------------------+---------------------+-------------+--------------------+-----------+-------------------+-------------------+-----------+-------------------+-------------------+-------------+-------------------+-------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+------------------+-----------------+-----------------+-------------------+---------------------+---------------------+---------------------+---------------------+-----------------------+-----------------------+----------------+------------------+------------------+--------------------+----------------------+-----------------

31121846

## Modeling

In [5]:
# Using 2020-2023 as train data and 2024 as test data to preserve order
train_data = df.filter(col("transit_timestamp") < "2024-01-01")
test_data = df.filter(col("transit_timestamp") >= "2024-01-01")

In [6]:
# Feature Engineering
feature_columns = [col for col in df.columns if col not in {"transit_timestamp", "ridership", "transfer"}]
print("Feature columns:", feature_columns)

Feature columns: ['transit_mode_index', 'station_complex_index', 'borough_index', 'payment_method_index', 'hour_of_day', 'hour_of_day_sin', 'hour_of_day_cos', 'day_of_week', 'day_of_week_sin', 'day_of_week_cos', 'week_of_month', 'week_of_month_sin', 'week_of_month_cos', 'ridership_lag_1', 'ridership_lag_2', 'ridership_lag_3', 'ridership_lag_4', 'ridership_lag_5', 'ridership_lag_6', 'ridership_lag_7', 'ridership_lag_8', 'ridership_lag_9', 'ridership_lag_10', 'ridership_lag_11', 'ridership_lag_12', 'ridership_lag_13', 'ridership_lag_14', 'ridership_lag_15', 'ridership_lag_16', 'ridership_lag_17', 'ridership_lag_18', 'ridership_lag_19', 'ridership_lag_20', 'ridership_lag_21', 'ridership_lag_22', 'ridership_lag_23', 'ridership_lag_24', 'ridership_7d_mv', 'hour_of_day_7d_mv', 'day_of_week_7d_mv', 'week_of_month_7d_mv', 'hour_of_day_sin_7d_mv', 'hour_of_day_cos_7d_mv', 'day_of_week_sin_7d_mv', 'day_of_week_cos_7d_mv', 'week_of_month_sin_7d_mv', 'week_of_month_cos_7d_mv', 'ridership_30d_mv', 

### XGBoost model

In [7]:
# XGB
# Assemble features into a single vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Train the XGBoost Model
xgb_regressor = SparkXGBRegressor(
    features_col="features",
    label_col="ridership",    
    num_workers=8,        # Adjusted based on cluster configuration
    max_depth=10,             
    learning_rate=0.1,         
)

# Create pipeline to chain feature assembly and model training
xgb_pipeline = Pipeline(stages=[assembler, xgb_regressor])

# Fit the model directly (without cross-validation)
xgb_model = xgb_pipeline.fit(train_data)

# Make Predictions
xgb_predictions = xgb_model.transform(test_data)

2025-04-18 09:40:15,218 INFO XGBoost-PySpark: _fit Running xgboost-3.0.0 on 8 workers with
	booster params: {'objective': 'reg:squarederror', 'device': 'cpu', 'learning_rate': 0.1, 'max_depth': 10, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}


2025-04-18 09:43:32,537 INFO XGBoost-PySpark: _train_booster Training on CPUs 8]
[09:43:33] Task 1 got rank 1
[09:43:33] Task 2 got rank 2
[09:43:33] Task 3 got rank 3
[09:43:33] Task 4 got rank 4
[09:43:33] Task 5 got rank 5
[09:43:33] Task 0 got rank 0
[09:43:33] Task 6 got rank 6
[09:43:33] Task 7 got rank 7
[09:43:47] [0]	training-rmse:259.06461
[09:43:48] [1]	training-rmse:234.54782
[09:43:49] [2]	training-rmse:212.58446
[09:43:50] [3]	training-rmse:192.85340
[09:43:51] [4]	training-rmse:175.21724
[09:43:52] [5]	training-rmse:159.39828
[09:43:53] [6]	training-rmse:145.30428
[09:43:54] [7]	training-rmse:132.71428
[09:43:55] [8]	training-rmse:121.48166
[09:43:56] [9]	training-rmse:111.48630
[09:43:56] [10]	training-rmse:102.60943
[09:43:57] [11]	training-rmse:94.69775
[09:43:58] [12]	training-rmse:87.69811
[09:43:59] [13]	training-rmse:81.57838
[09:44:00] [14]	training-rmse:76.13552
[09:44:01] [15]	training-rmse:71.37346
[09:44:02] [16]	training-rmse:67.20486
[09:44:03] [17]	trainin

In [8]:
# XGB metrics
xgb_metrics = {
    'rmse': 'Root Mean Squared Error',
    'mse': 'Mean Squared Error',
    'mae': 'Mean Absolute Error'
}

for metric, name in xgb_metrics.items():
    xgb_evaluator = RegressionEvaluator(
        labelCol="ridership",
        predictionCol="prediction",
        metricName=metric)
    value = xgb_evaluator.evaluate(xgb_predictions)
    print(f"{name} ({metric.upper()}): {value}")

# Root Mean Squared Error (RMSE): 73.01
# Mean Squared Error (MSE): 5330.71
# Mean Absolute Error (MAE): 21.41

2025-04-18 00:29:10,664 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


Root Mean Squared Error (RMSE): 73.0117556600609


2025-04-18 00:32:23,732 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


Mean Squared Error (MSE): 5330.716464564435


2025-04-18 00:35:36,089 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


Mean Absolute Error (MAE): 21.413205062840547


#### Save model in cloud

In [8]:
# Save the trained XGBoost model to GCS
gcs_model_path = "gs://model_stored_for_pred_forecast/hourly_prediction"
xgb_model.write().overwrite().save(gcs_model_path)

### Random Forest model

In [ ]:
# RF model
# Assemble features into a single vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Train the Random Forest Model
rf_regressor = RandomForestRegressor(
    featuresCol="features",
    labelCol="ridership",
    numTrees=100,                 # Number of trees in the forest
    maxDepth=10,                  # Maximum depth of each tree
    featureSubsetStrategy="auto", # Number of features to consider for splits
    subsamplingRate=1.0,          # Fraction of training data used for each tree
    seed=42                       
)

# Create a pipeline to chain feature assembly and model training
rf_pipeline = Pipeline(stages=[assembler, rf_regressor])

# Fit the model
rf_model = rf_pipeline.fit(train_data)

# Make Predictions
rf_predictions = rf_model.transform(test_data)

In [ ]:
# RF metrics
rf_metrics = {
    'rmse': 'Root Mean Squared Error',
    'mse': 'Mean Squared Error',
    'mae': 'Mean Absolute Error'
}

for metric, name in rf_metrics.items():
    rf_evaluator = RegressionEvaluator(
        labelCol="ridership",
        predictionCol="prediction",
        metricName=metric)
    value = rf_evaluator.evaluate(rf_predictions)
    print(f"{name} ({metric.upper()}): {value}")

# Root Mean Squared Error (RMSE): 108.04
# Mean Squared Error (MSE): 11673.61
# Mean Absolute Error (MAE): 29.02

### Storing best performing model's predictions to BigQuery

In [14]:
# Convert timestamp_ntz datatype to timestamp
xgb_predictions = xgb_predictions.withColumn("transit_timestamp", col("transit_timestamp").cast("timestamp"))
xgb_predictions.show(5, truncate=False)

+-------------------+---------+--------+------------------+---------------------+-------------+--------------------+-----------+------------------+--------------------+-----------+-------------------+------------------+-------------+-------------------+-------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+------------------+-----------------+-----------------+-------------------+---------------------+---------------------+---------------------+---------------------+-----------------------+-----------------------+------------------+------------------+------------------+--------------------+----------------------+----------------

2025-04-18 00:38:49,654 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


In [15]:
# Filter relevant columns
xgb_hour_pred = xgb_predictions.select("transit_timestamp", "transit_mode_index", "borough_index", "station_complex_index", "payment_method_index", "ridership", "transfer", "prediction")
xgb_hour_pred.show(5, truncate=False)

+-------------------+------------------+-------------+---------------------+--------------------+---------+--------+------------------+
|transit_timestamp  |transit_mode_index|borough_index|station_complex_index|payment_method_index|ridership|transfer|prediction        |
+-------------------+------------------+-------------+---------------------+--------------------+---------+--------+------------------+
|2024-01-23 07:00:00|2                 |4            |183                  |1                   |122      |0       |105.78429412841797|
|2024-01-23 07:00:00|2                 |2            |107                  |2                   |227      |6       |234.78582763671875|
|2024-01-23 08:00:00|2                 |4            |5                    |2                   |891      |67      |871.6275634765625 |
|2024-01-23 08:00:00|2                 |2            |406                  |1                   |306      |1       |267.4712829589844 |
|2024-01-23 08:00:00|2                 |3       

2025-04-18 00:39:01,741 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


In [16]:
# Map encoded categorical columns to their original values
payment_method_mapping = {1: 'metrocard', 2: 'omny'}
transit_mode_mapping = {1: 'staten_island_railway', 2: 'subway', 3: 'tram'}
borough_mapping = {1: 'Bronx', 2: 'Brooklyn', 3: 'Manhattan', 4: 'Queens', 5: 'Staten Island'}

station_df = spark.read.csv("station_complex.csv", header=True)
station_df = station_df.withColumn("station_complex_index", col("station_complex_index").cast("int"))
station_df = station_df.select("station_complex_index", "station_complex")
# station_df.printSchema()

station_map = station_df.rdd.collectAsMap()
# print(station_mapping)
station_mapping = [
    item for pair in station_map.items() for item in (lit(pair[0]), lit(pair[1]))
]

# Create mapping expressions
payment_method_expr = create_map([lit(k) for pair in payment_method_mapping.items() for k in pair])
transit_mode_expr = create_map([lit(k) for pair in transit_mode_mapping.items() for k in pair])
borough_expr = create_map([lit(k) for pair in borough_mapping.items() for k in pair])
station_complex_expr = create_map(*station_mapping)

# Apply the mappings
xgb_hour_pred_mapped = xgb_hour_pred.withColumn(
    "payment_method", 
    payment_method_expr[col("payment_method_index").cast("int")]
).withColumn(
    "transit_mode", 
    transit_mode_expr[col("transit_mode_index").cast("int")]
).withColumn(
    "borough", 
    borough_expr[col("borough_index").cast("int")]
).withColumn(
    "station_complex", 
    station_complex_expr[col("station_complex_index").cast("int")]
)

xgb_hour_pred_mapped.show(5, truncate=False)

+-------------------+------------------+-------------+---------------------+--------------------+---------+--------+------------------+--------------+------------+---------+-----------------------+
|transit_timestamp  |transit_mode_index|borough_index|station_complex_index|payment_method_index|ridership|transfer|prediction        |payment_method|transit_mode|borough  |station_complex        |
+-------------------+------------------+-------------+---------------------+--------------------+---------+--------+------------------+--------------+------------+---------+-----------------------+
|2024-01-23 07:00:00|2                 |4            |183                  |1                   |122      |0       |105.78429412841797|metrocard     |subway      |Queens   |Beach 60 St (A)        |
|2024-01-23 07:00:00|2                 |2            |107                  |2                   |227      |6       |234.78582763671875|omny          |subway      |Brooklyn |53 St (R)              |
|2024-01-2

2025-04-18 00:39:14,563 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


In [17]:
# Filter relevant columns
xgb_hour_pred_bq = xgb_hour_pred_mapped.select('transit_timestamp', 'transit_mode', 'borough', 'station_complex', 'payment_method', 'ridership', 'transfer', 'prediction')
xgb_hour_pred_bq.show(5)

+-------------------+------------+---------+--------------------+--------------+---------+--------+------------------+
|  transit_timestamp|transit_mode|  borough|     station_complex|payment_method|ridership|transfer|        prediction|
+-------------------+------------+---------+--------------------+--------------+---------+--------+------------------+
|2024-01-23 07:00:00|      subway|   Queens|     Beach 60 St (A)|     metrocard|      122|       0|105.78429412841797|
|2024-01-23 07:00:00|      subway| Brooklyn|           53 St (R)|          omny|      227|       6|234.78582763671875|
|2024-01-23 08:00:00|      subway|   Queens|103 St-Corona Pla...|          omny|      891|      67| 871.6275634765625|
|2024-01-23 08:00:00|      subway| Brooklyn|        Union St (R)|     metrocard|      306|       1| 267.4712829589844|
|2024-01-23 08:00:00|      subway|Manhattan|     Prince St (R,W)|          omny|      175|       0|107.21585083007812|
+-------------------+------------+---------+----

2025-04-18 00:39:27,027 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


### Store predictions in BigQuery

In [18]:
# Write predictions to BigQuery
xgb_hour_pred_bq.write.format("bigquery") \
    .option("table", "lively-encoder-448916-d5.nyc_subway.hourly_predictions") \
    .option("temporaryGcsBucket", "temp_nyc_bucket_for_bq") \
    .mode("overwrite") \
    .save()

2025-04-18 00:39:44,571 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
